路由

In [7]:
import io
import contextlib 
from ChatGLM3_router import ChatGLM
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [8]:
# 主要创建三个知识域的模板，一个营销文案作者，一个客服，一个小红书

Copywriter_template = """You are a master of persuasion and a pro at captivating audiences. \
You have the ability to craft captivating messages that move people to action. \
You know that the key to effective communication is understanding your audience and speaking directly to their needs and desires.\

Here is a question:
{input}"""

RedBookWriter_template ="""You are a beauty blogger with a knack for discovering the latest trends and providing makeup tips. \
You have an eye for aesthetics and are always looking for ways to enhance your beauty routine. \
You are passionate about sharing your knowledge and inspiring others to embrace their beauty. 

Here is a question:
{input}"""

customerService_template = """You are a customer service representative. You are skilled at assisting customers and addressing their concerns. \
You take a problem-solving approach to help resolve issues and provide solutions. You should answer product, brand questions and purchase questions，and introduce product features.

Here is a question:
{input}"""

In [9]:
prompt_infos = [
    {
        "name": "Copywriter",
        "description": "Good for writing marketing copy ",
        "prompt_template": Copywriter_template,
    },
    {
        "name": "customerService",
        "description": "Good for answering product, brand, purchase and product features questions",
        "prompt_template": customerService_template,
    },
    {
        "name": "RedBookWriter",
        "description": "Good for writing Script title soft article advertising Weibo tweet ",
        "prompt_template": RedBookWriter_template,
    },
]

In [10]:
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
llm.chat([{"role":"user","content":"你好"}])

KeyboardInterrupt: 

In [6]:
obj = llm.stream_chat({'role':'user','content':'你好'})
for i in obj:
    print(obj)


<generator object ChatGLM.stream_chat at 0x7fcabd0d89e0>

In [9]:
# 下面这段主要通过遍历得到destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
# 得到一个起始chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [10]:
'''
#使用cohere
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
import os
os.environ["COHERE_API_KEY"]="JaYT27nDgJYh6yvsbknQdW4yYLuMTvNsvvBy7HVl"

52/27
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]

names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),
]
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]
)
'''

'\n#使用cohere\nfrom langchain.chains.router.embedding_router import EmbeddingRouterChain\nfrom langchain.embeddings import CohereEmbeddings\nfrom langchain.vectorstores import Chroma\nimport os\nos.environ["COHERE_API_KEY"]="JaYT27nDgJYh6yvsbknQdW4yYLuMTvNsvvBy7HVl"\n\n52/27\nnames_and_descriptions = [\n    ("Copywriter", ["营销文案写作任务"]),\n    ("customerService", ["知识问答产品选择任务"]),\n    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),\n]\n\nnames_and_descriptions = [\n    ("Copywriter", ["营销文案写作任务"]),\n    ("customerService", ["知识问答产品选择任务"]),\n    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),\n]\nrouter_chain = EmbeddingRouterChain.from_names_and_descriptions(\n    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]\n)\n'

In [11]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.vectorstores import Chroma
'''
47/31
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]
'''
'''
35/27
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),
]
'''
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embed_model = LangchainEmbedding(embeddings)
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, embed_model, routing_keys=["input"]
)



In [12]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [13]:
import pandas as pd
df = pd.read_excel("/home/kit/kit/ChatGLM3/langchain_demo/NewPromptRouter.xlsx",header=None)
df = df.assign(direct = None,extract = None)
df

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,None,None
...,...,...,...
115,帮我写个文案，小红书风格的，汤臣倍健BYHEALTH品牌的益生菌，含有益微生物，可以保证肠道...,None,None
116,帮我写个文案，小红书风格的，乐力Osteoform品牌的益生菌，主要用于腹泻腹痛、受凉感冒、...,None,None
117,帮我写个文案，小红书风格的，迪辅乐Dipro品牌的益生菌，改善乳糖不耐受、改善肠道菌群结构、...,None,None
118,帮我写个文案，小红书风格的，圣歌兰Life's Flora品牌的益生菌，可以降低胆固醇、预防...,None,None


In [14]:
#输入user_question,判断句子路由方向，返回1代表生成文案，返回0代表知识问答
def get_direct(user_question,chain):
    with contextlib.redirect_stdout(io.StringIO()) as f:
        print(
            chain.run(
                user_question
            )
        )
        ans = f.getvalue() 
    sign = ans[51:61]
    if sign == "Copywriter":
        return 1
    else:
        return 0

In [15]:
for i in range(int(len(df))):
        direct = get_direct(df.iloc[i,0],chain)
        df.iloc[i,1] = direct
df

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,1,None
...,...,...,...
115,帮我写个文案，小红书风格的，汤臣倍健BYHEALTH品牌的益生菌，含有益微生物，可以保证肠道...,0,None
116,帮我写个文案，小红书风格的，乐力Osteoform品牌的益生菌，主要用于腹泻腹痛、受凉感冒、...,1,None
117,帮我写个文案，小红书风格的，迪辅乐Dipro品牌的益生菌，改善乳糖不耐受、改善肠道菌群结构、...,1,None
118,帮我写个文案，小红书风格的，圣歌兰Life's Flora品牌的益生菌，可以降低胆固醇、预防...,1,None


In [16]:
df_ex = df[df['direct'] != 0].reset_index(drop=True)
df_ex

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,1,None
5,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： 潘多拉新款项链\n\n...,1,None
6,你是一名美食博主，请根据以下要求创作一篇小红书种草笔记： 文章主题：潮汕牛肉火锅探店的感受和...,1,None
7,你是一名美食博主，请根据以下要求创作一篇小红书种草笔记： 文章主题：韩式烤肉探店的感受和体验...,1,None
8,你是一个美妆博主，需要根据季节推荐适合的护肤品。\n\n季节： 夏季 \n\n皮肤类型： 干...,1,None
9,你是一个美妆博主，需要根据季节推荐适合的护肤品。\n\n季节： 秋季 \n\n皮肤类型： 干...,1,None


路由结果（32/61）错误路由29个
抽取

In [ ]:
import torch
import time
import math
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [ ]:
def init_model():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer


model, tokenizer = init_model()
model.eval()

In [ ]:
prompt = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
可以返回空
帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的
"""
for i in range(len(df_ex)):
    user_question = df_ex.iloc[i,0]
    input = prompt.replace("帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的",user_question)
    response,history = model.chat(tokenizer, input, history=[])
    print(i,input,response)
    df_ex.iloc[i,2] = response

In [ ]:
df_ex.to_excel("./outputRouterNR.xlsx")